In [173]:
pip install psycopg2

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [174]:
import pandas as pd
import psycopg2
import os
import json

In [176]:
DB_ARGS = {
    'db': 'home_credit',
    'user': os.getenv('DB_USER'),
    'host': '127.0.0.1',    
    'password': os.getenv('DB_PASSWORD')
}

In [177]:
DATA_FULL_PATH = "C:\\Program Files\\PostgreSQL\\14\\data\\"

In [178]:
class MyDatabase:
    """
    Класс представляет собой соединение к базе данных, 
    через которое можно отрпавлять запросы и получать ответы
    
    :param db:  название БД.
    :type db: str
    :param user: имя пользователя БД.
    :type user: str
    :param host: хост для подключения к БД.
    :type host: str
    :param password: парль пользователя БД.
    :type password: str
    """
    def __init__(self, db="mydb", user="postgres", host="127.0.0.1", password="postgres"):
        self.conn = psycopg2.connect(dbname=db, 
                                     user=user, 
                                     host=host, 
                                     password=password
                                    )
        self.cur = self.conn.cursor()

    def send_query(self, query):
        """
        Выполняет запрос к базе.

        :param query: строка с sql запросом.
        """
        try:      
            self.cur.execute(query)
            self.conn.commit()
        except (Exception, psycopg2.Error) as error:
            print("Error while fetching data from PostgreSQL", error)
        
        
    def get_query(self, query):
        """
        Выполняет запрос к базе, который должен вернуть результат

        :param query: строка с sql запросом.
        
        :return: pandas.DataFrame
        """
        df = pd.read_sql(query, self.conn)
        return df

    def close(self):
        """
        Закрывает соединение с БД.
        """
        self.cur.close()
        self.conn.close()


Пример работы с классом

In [179]:
db = MyDatabase(**DB_ARGS)
df = db.get_query("SELECT SK_ID_CURR FROM bureau;")
db.close()
df

OperationalError: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
postgres_db = MyDatabase(**DB_ARGS)

In [ ]:
# bureau
sql_schema_query = """
    DROP TABLE IF EXISTS bureau; 
    CREATE TABLE bureau(
      SK_ID_CURR integer,
      SK_ID_BUREAU integer,
      CREDIT_ACTIVE varchar(13),
      CREDIT_CURRENCY varchar(15),
      DAYS_CREDIT smallint,
      CREDIT_DAY_OVERDUE smallint,
      DAYS_CREDIT_ENDDATE real,
      DAYS_ENDDATE_FACT real,
      AMT_CREDIT_MAX_OVERDUE real,
      CNT_CREDIT_PROLONG smallint,
      AMT_CREDIT_SUM real,
      AMT_CREDIT_SUM_DEBT real,
      AMT_CREDIT_SUM_LIMIT real,
      AMT_CREDIT_SUM_OVERDUE real,
      CREDIT_TYPE varchar(49),
      DAYS_CREDIT_UPDATE integer,
      AMT_ANNUITY  real)
"""
sql_data_query = f"""
COPY bureau(
  SK_ID_CURR,
  SK_ID_BUREAU,
  CREDIT_ACTIVE,
  CREDIT_CURRENCY,
  DAYS_CREDIT,
  CREDIT_DAY_OVERDUE,
  DAYS_CREDIT_ENDDATE,
  DAYS_ENDDATE_FACT,
  AMT_CREDIT_MAX_OVERDUE,
  CNT_CREDIT_PROLONG,
  AMT_CREDIT_SUM,
  AMT_CREDIT_SUM_DEBT,
  AMT_CREDIT_SUM_LIMIT,
  AMT_CREDIT_SUM_OVERDUE,
  CREDIT_TYPE,
  DAYS_CREDIT_UPDATE,
  AMT_ANNUITY
)
FROM '{DATA_FULL_PATH}bureau.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# bureau_balance
sql_schema_query = """
    DROP TABLE IF EXISTS bureau_balance; 
    CREATE TABLE bureau_balance(
      SK_ID_BUREAU integer,
      MONTHS_BALANCE smallint,
      STATUS varchar(5))
"""

sql_data_query = f"""
COPY bureau_balance(
   SK_ID_BUREAU,
   MONTHS_BALANCE,
   STATUS
)
FROM '{DATA_FULL_PATH}bureau_balance.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# credit_card_balance
sql_schema_query = """
    DROP TABLE IF EXISTS credit_card_balance; 
    CREATE TABLE credit_card_balance(
      SK_ID_PREV integer,
      SK_ID_CURR integer,
      MONTHS_BALANCE smallint,
      AMT_BALANCE real,
      AMT_CREDIT_LIMIT_ACTUAL integer,
      AMT_DRAWINGS_ATM_CURRENT real,
      AMT_DRAWINGS_CURRENT real,
      AMT_DRAWINGS_OTHER_CURRENT real,
      AMT_DRAWINGS_POS_CURRENT real,
      AMT_INST_MIN_REGULARITY real,
      AMT_PAYMENT_CURRENT real,
      AMT_PAYMENT_TOTAL_CURRENT real,
      AMT_RECEIVABLE_PRINCIPAL real,
      AMT_RECIVABLE real,
      AMT_TOTAL_RECEIVABLE real,
      CNT_DRAWINGS_ATM_CURRENT real,
      CNT_DRAWINGS_CURRENT smallint,
      CNT_DRAWINGS_OTHER_CURRENT real,
      CNT_DRAWINGS_POS_CURRENT real,
      CNT_INSTALMENT_MATURE_CUM real,
      NAME_CONTRACT_STATUS varchar(15),
      SK_DPD smallint,
      SK_DPD_DEF smallint)
"""

sql_data_query = f"""
COPY credit_card_balance(
      SK_ID_PREV,
      SK_ID_CURR,
      MONTHS_BALANCE,
      AMT_BALANCE,
      AMT_CREDIT_LIMIT_ACTUAL,
      AMT_DRAWINGS_ATM_CURRENT,
      AMT_DRAWINGS_CURRENT,
      AMT_DRAWINGS_OTHER_CURRENT,
      AMT_DRAWINGS_POS_CURRENT,
      AMT_INST_MIN_REGULARITY,
      AMT_PAYMENT_CURRENT,
      AMT_PAYMENT_TOTAL_CURRENT,
      AMT_RECEIVABLE_PRINCIPAL,
      AMT_RECIVABLE,
      AMT_TOTAL_RECEIVABLE,
      CNT_DRAWINGS_ATM_CURRENT,
      CNT_DRAWINGS_CURRENT,
      CNT_DRAWINGS_OTHER_CURRENT,
      CNT_DRAWINGS_POS_CURRENT,
      CNT_INSTALMENT_MATURE_CUM,
      NAME_CONTRACT_STATUS,
      SK_DPD,
      SK_DPD_DEF
)
FROM '{DATA_FULL_PATH}credit_card_balance.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# installments_payments
sql_schema_query = """
    DROP TABLE IF EXISTS installments_payments; 
    CREATE TABLE installments_payments(
      SK_ID_PREV integer,
      SK_ID_CURR integer,
      NUM_INSTALMENT_VERSION real,
      NUM_INSTALMENT_NUMBER smallint,
      DAYS_INSTALMENT real,
      DAYS_ENTRY_PAYMENT real,
      AMT_INSTALMENT real,
      AMT_PAYMENT real)
"""

sql_data_query = f"""
COPY installments_payments(
      SK_ID_PREV,
      SK_ID_CURR,
      NUM_INSTALMENT_VERSION,
      NUM_INSTALMENT_NUMBER,
      DAYS_INSTALMENT,
      DAYS_ENTRY_PAYMENT,
      AMT_INSTALMENT,
      AMT_PAYMENT
)
FROM '{DATA_FULL_PATH}installments_payments.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# Таблица POS_CASH_balance
sql_schema_query = """
    DROP TABLE IF EXISTS POS_CASH_balance; 
    CREATE TABLE POS_CASH_balance(
      SK_ID_PREV integer,
      SK_ID_CURR integer,
      MONTHS_BALANCE smallint,
      CNT_INSTALMENT real,
      CNT_INSTALMENT_FUTURE real,
      NAME_CONTRACT_STATUS varchar(25),
      SK_DPD smallint,
      SK_DPD_DEF smallint)
"""

sql_data_query = f"""
COPY POS_CASH_balance(
      SK_ID_PREV,
      SK_ID_CURR,
      MONTHS_BALANCE,
      CNT_INSTALMENT,
      CNT_INSTALMENT_FUTURE,
      NAME_CONTRACT_STATUS,
      SK_DPD,
      SK_DPD_DEF
)
FROM '{DATA_FULL_PATH}POS_CASH_balance.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# Таблица previous_application
sql_schema_query = """
    DROP TABLE IF EXISTS previous_application; 
    CREATE TABLE previous_application(
      SK_ID_PREV integer,
      SK_ID_CURR integer,
      NAME_CONTRACT_TYPE varchar(20),
      AMT_ANNUITY real,
      AMT_APPLICATION real,
      AMT_CREDIT real,
      AMT_DOWN_PAYMENT real,
      AMT_GOODS_PRICE real,
      WEEKDAY_APPR_PROCESS_START varchar(10),
      HOUR_APPR_PROCESS_START smallint,
      FLAG_LAST_APPL_PER_CONTRACT varchar(5),
      NFLAG_LAST_APPL_IN_DAY smallint,
      RATE_DOWN_PAYMENT real,
      RATE_INTEREST_PRIMARY real,
      RATE_INTEREST_PRIVILEGED real,
      NAME_CASH_LOAN_PURPOSE varchar(40),
      NAME_CONTRACT_STATUS varchar(15),
      DAYS_DECISION smallint,
      NAME_PAYMENT_TYPE varchar(45),
      CODE_REJECT_REASON varchar(10),
      NAME_TYPE_SUITE varchar(17),
      NAME_CLIENT_TYPE varchar(10),
      NAME_GOODS_CATEGORY varchar(30),
      NAME_PORTFOLIO varchar(7),
      NAME_PRODUCT_TYPE varchar(10),
      CHANNEL_TYPE varchar(30),
      SELLERPLACE_AREA integer,
      NAME_SELLER_INDUSTRY varchar(25),
      CNT_PAYMENT real,
      NAME_YIELD_GROUP varchar(13),
      PRODUCT_COMBINATION varchar(35),
      DAYS_FIRST_DRAWING real,
      DAYS_FIRST_DUE real,
      DAYS_LAST_DUE_1ST_VERSION real,
      DAYS_LAST_DUE real,
      DAYS_TERMINATION real,
      NFLAG_INSURED_ON_APPROVAL real)
"""

sql_data_query = f"""
COPY previous_application(
      SK_ID_PREV,
      SK_ID_CURR,
      NAME_CONTRACT_TYPE,
      AMT_ANNUITY,
      AMT_APPLICATION,
      AMT_CREDIT,
      AMT_DOWN_PAYMENT,
      AMT_GOODS_PRICE,
      WEEKDAY_APPR_PROCESS_START,
      HOUR_APPR_PROCESS_START,
      FLAG_LAST_APPL_PER_CONTRACT,
      NFLAG_LAST_APPL_IN_DAY,
      RATE_DOWN_PAYMENT,
      RATE_INTEREST_PRIMARY,
      RATE_INTEREST_PRIVILEGED,
      NAME_CASH_LOAN_PURPOSE,
      NAME_CONTRACT_STATUS,
      DAYS_DECISION,
      NAME_PAYMENT_TYPE,
      CODE_REJECT_REASON,
      NAME_TYPE_SUITE,
      NAME_CLIENT_TYPE,
      NAME_GOODS_CATEGORY,
      NAME_PORTFOLIO,
      NAME_PRODUCT_TYPE,
      CHANNEL_TYPE,
      SELLERPLACE_AREA,
      NAME_SELLER_INDUSTRY,
      CNT_PAYMENT,
      NAME_YIELD_GROUP,
      PRODUCT_COMBINATION,
      DAYS_FIRST_DRAWING,
      DAYS_FIRST_DUE,
      DAYS_LAST_DUE_1ST_VERSION,
      DAYS_LAST_DUE,
      DAYS_TERMINATION,
      NFLAG_INSURED_ON_APPROVAL
)
FROM '{DATA_FULL_PATH}previous_application.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# Таблица sample_submission
sql_schema_query = """
    DROP TABLE IF EXISTS sample_submission; 
    CREATE TABLE sample_submission(
      SK_ID_CURR integer,
      TARGET real)
"""

sql_data_query = f"""
COPY sample_submission(
      SK_ID_CURR,
      TARGET
)
FROM '{DATA_FULL_PATH}sample_submission.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# Таблица application_test
sql_schema_query = """
    DROP TABLE IF EXISTS application_test; 
    CREATE TABLE application_test(
        SK_ID_CURR integer,
        NAME_CONTRACT_TYPE varchar(17),
        CODE_GENDER varchar(3),
        FLAG_OWN_CAR char(1),
        FLAG_OWN_REALTY char(1),
        CNT_CHILDREN real,
        AMT_INCOME_TOTAL real,
        AMT_CREDIT real,
        AMT_ANNUITY real,
        AMT_GOODS_PRICE real,
        NAME_TYPE_SUITE varchar(17),
        NAME_INCOME_TYPE varchar(25),
        NAME_EDUCATION_TYPE varchar(30),
        NAME_FAMILY_STATUS varchar(26),
        NAME_HOUSING_TYPE varchar(25),
        REGION_POPULATION_RELATIVE real,
        DAYS_BIRTH integer,
        DAYS_EMPLOYED integer,
        DAYS_REGISTRATION real,
        DAYS_ID_PUBLISH smallint,
        OWN_CAR_AGE real,
        FLAG_MOBIL boolean,
        FLAG_EMP_PHONE boolean,
        FLAG_WORK_PHONE boolean,
        FLAG_CONT_MOBILE boolean,
        FLAG_PHONE boolean,
        FLAG_EMAIL boolean,
        OCCUPATION_TYPE varchar(40),
        CNT_FAM_MEMBERS real,
        REGION_RATING_CLIENT smallint, 
        REGION_RATING_CLIENT_W_CITY smallint,
        WEEKDAY_APPR_PROCESS_START varchar(9),
        HOUR_APPR_PROCESS_START smallint,
        REG_REGION_NOT_LIVE_REGION boolean,
        REG_REGION_NOT_WORK_REGION boolean,
        LIVE_REGION_NOT_WORK_REGION boolean,
        REG_CITY_NOT_LIVE_CITY boolean,
        REG_CITY_NOT_WORK_CITY boolean,
        LIVE_CITY_NOT_WORK_CITY boolean,
        ORGANIZATION_TYPE varchar(30), 
        EXT_SOURCE_1 real,
        EXT_SOURCE_2 real,
        EXT_SOURCE_3 real,
        APARTMENTS_AVG real,
        BASEMENTAREA_AVG real,
        YEARS_BEGINEXPLUATATION_AVG real,
        YEARS_BUILD_AVG real,
        COMMONAREA_AVG real,
        ELEVATORS_AVG real,
        ENTRANCES_AVG real,
        FLOORSMAX_AVG real,
        FLOORSMIN_AVG real,
        LANDAREA_AVG real,
        LIVINGAPARTMENTS_AVG real,
        LIVINGAREA_AVG real,
        NONLIVINGAPARTMENTS_AVG real,
        NONLIVINGAREA_AVG real,
        APARTMENTS_MODE real,
        BASEMENTAREA_MODE real,
        YEARS_BEGINEXPLUATATION_MODE real,
        YEARS_BUILD_MODE real,
        COMMONAREA_MODE real,
        ELEVATORS_MODE real,
        ENTRANCES_MODE real,
        FLOORSMAX_MODE real,
        FLOORSMIN_MODE real,
        LANDAREA_MODE real,
        LIVINGAPARTMENTS_MODE real,
        LIVINGAREA_MODE real,
        NONLIVINGAPARTMENTS_MODE real,
        NONLIVINGAREA_MODE real,
        APARTMENTS_MEDI real,
        BASEMENTAREA_MEDI real,
        YEARS_BEGINEXPLUATATION_MEDI real,
        YEARS_BUILD_MEDI real,
        COMMONAREA_MEDI real,
        ELEVATORS_MEDI real,
        ENTRANCES_MEDI real,
        FLOORSMAX_MEDI real,
        FLOORSMIN_MEDI real,
        LANDAREA_MEDI real,
        LIVINGAPARTMENTS_MEDI real,
        LIVINGAREA_MEDI real,
        NONLIVINGAPARTMENTS_MEDI real,
        NONLIVINGAREA_MEDI real,
        FONDKAPREMONT_MODE varchar(25),
        HOUSETYPE_MODE varchar(17),
        TOTALAREA_MODE real,
        WALLSMATERIAL_MODE varchar(13),
        EMERGENCYSTATE_MODE varchar(3),
        OBS_30_CNT_SOCIAL_CIRCLE real,
        DEF_30_CNT_SOCIAL_CIRCLE real,
        OBS_60_CNT_SOCIAL_CIRCLE real,
        DEF_60_CNT_SOCIAL_CIRCLE real,
        DAYS_LAST_PHONE_CHANGE real,
        FLAG_DOCUMENT_2 boolean,
        FLAG_DOCUMENT_3 boolean,
        FLAG_DOCUMENT_4 boolean,
        FLAG_DOCUMENT_5 boolean,
        FLAG_DOCUMENT_6 boolean,
        FLAG_DOCUMENT_7 boolean,
        FLAG_DOCUMENT_8 boolean,
        FLAG_DOCUMENT_9 boolean,
        FLAG_DOCUMENT_10 boolean,
        FLAG_DOCUMENT_11 boolean,
        FLAG_DOCUMENT_12 boolean,
        FLAG_DOCUMENT_13 boolean,
        FLAG_DOCUMENT_14 boolean,
        FLAG_DOCUMENT_15 boolean,
        FLAG_DOCUMENT_16 boolean,
        FLAG_DOCUMENT_17 boolean,
        FLAG_DOCUMENT_18 boolean,
        FLAG_DOCUMENT_19 boolean,
        FLAG_DOCUMENT_20 boolean,
        FLAG_DOCUMENT_21 boolean,
        AMT_REQ_CREDIT_BUREAU_HOUR real,
        AMT_REQ_CREDIT_BUREAU_DAY real,
        AMT_REQ_CREDIT_BUREAU_WEEK real,
        AMT_REQ_CREDIT_BUREAU_MON real,
        AMT_REQ_CREDIT_BUREAU_QRT real,
        AMT_REQ_CREDIT_BUREAU_YEAR real)
"""

sql_data_query = f"""
COPY application_test(
    SK_ID_CURR,
    NAME_CONTRACT_TYPE,
    CODE_GENDER,
    FLAG_OWN_CAR,
    FLAG_OWN_REALTY,
    CNT_CHILDREN,
    AMT_INCOME_TOTAL,
    AMT_CREDIT,
    AMT_ANNUITY,
    AMT_GOODS_PRICE,
    NAME_TYPE_SUITE,
    NAME_INCOME_TYPE,
    NAME_EDUCATION_TYPE,
    NAME_FAMILY_STATUS,
    NAME_HOUSING_TYPE,
    REGION_POPULATION_RELATIVE,
    DAYS_BIRTH,
    DAYS_EMPLOYED,
    DAYS_REGISTRATION,
    DAYS_ID_PUBLISH,
    OWN_CAR_AGE,
    FLAG_MOBIL,
    FLAG_EMP_PHONE,
    FLAG_WORK_PHONE,
    FLAG_CONT_MOBILE,
    FLAG_PHONE,
    FLAG_EMAIL,
    OCCUPATION_TYPE,
    CNT_FAM_MEMBERS,
    REGION_RATING_CLIENT,
    REGION_RATING_CLIENT_W_CITY,
    WEEKDAY_APPR_PROCESS_START,
    HOUR_APPR_PROCESS_START,
    REG_REGION_NOT_LIVE_REGION,
    REG_REGION_NOT_WORK_REGION,
    LIVE_REGION_NOT_WORK_REGION,
    REG_CITY_NOT_LIVE_CITY,
    REG_CITY_NOT_WORK_CITY,
    LIVE_CITY_NOT_WORK_CITY,
    ORGANIZATION_TYPE,
    EXT_SOURCE_1,
    EXT_SOURCE_2,
    EXT_SOURCE_3,
    APARTMENTS_AVG,
    BASEMENTAREA_AVG,
    YEARS_BEGINEXPLUATATION_AVG,
    YEARS_BUILD_AVG,
    COMMONAREA_AVG,
    ELEVATORS_AVG,
    ENTRANCES_AVG,
    FLOORSMAX_AVG,
    FLOORSMIN_AVG,
    LANDAREA_AVG,
    LIVINGAPARTMENTS_AVG,
    LIVINGAREA_AVG,
    NONLIVINGAPARTMENTS_AVG,
    NONLIVINGAREA_AVG,
    APARTMENTS_MODE,
    BASEMENTAREA_MODE,
    YEARS_BEGINEXPLUATATION_MODE,
    YEARS_BUILD_MODE,
    COMMONAREA_MODE,
    ELEVATORS_MODE,
    ENTRANCES_MODE,
    FLOORSMAX_MODE,
    FLOORSMIN_MODE,
    LANDAREA_MODE,
    LIVINGAPARTMENTS_MODE,
    LIVINGAREA_MODE,
    NONLIVINGAPARTMENTS_MODE,
    NONLIVINGAREA_MODE,
    APARTMENTS_MEDI,
    BASEMENTAREA_MEDI,
    YEARS_BEGINEXPLUATATION_MEDI,
    YEARS_BUILD_MEDI,
    COMMONAREA_MEDI,
    ELEVATORS_MEDI,
    ENTRANCES_MEDI,
    FLOORSMAX_MEDI,
    FLOORSMIN_MEDI,
    LANDAREA_MEDI,
    LIVINGAPARTMENTS_MEDI,
    LIVINGAREA_MEDI,
    NONLIVINGAPARTMENTS_MEDI,
    NONLIVINGAREA_MEDI,
    FONDKAPREMONT_MODE,
    HOUSETYPE_MODE,
    TOTALAREA_MODE,
    WALLSMATERIAL_MODE,
    EMERGENCYSTATE_MODE,
    OBS_30_CNT_SOCIAL_CIRCLE,
    DEF_30_CNT_SOCIAL_CIRCLE,
    OBS_60_CNT_SOCIAL_CIRCLE,
    DEF_60_CNT_SOCIAL_CIRCLE,
    DAYS_LAST_PHONE_CHANGE,
    FLAG_DOCUMENT_2,
    FLAG_DOCUMENT_3,
    FLAG_DOCUMENT_4,
    FLAG_DOCUMENT_5,
    FLAG_DOCUMENT_6,
    FLAG_DOCUMENT_7,
    FLAG_DOCUMENT_8,
    FLAG_DOCUMENT_9,
    FLAG_DOCUMENT_10,
    FLAG_DOCUMENT_11,
    FLAG_DOCUMENT_12,
    FLAG_DOCUMENT_13,
    FLAG_DOCUMENT_14,
    FLAG_DOCUMENT_15,
    FLAG_DOCUMENT_16,
    FLAG_DOCUMENT_17,
    FLAG_DOCUMENT_18,
    FLAG_DOCUMENT_19,
    FLAG_DOCUMENT_20,
    FLAG_DOCUMENT_21,
    AMT_REQ_CREDIT_BUREAU_HOUR,
    AMT_REQ_CREDIT_BUREAU_DAY,
    AMT_REQ_CREDIT_BUREAU_WEEK,
    AMT_REQ_CREDIT_BUREAU_MON,
    AMT_REQ_CREDIT_BUREAU_QRT,
    AMT_REQ_CREDIT_BUREAU_YEAR
)
FROM '{DATA_FULL_PATH}application_test.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

In [ ]:
# Таблица application_train
sql_schema_query = """
    DROP TABLE IF EXISTS application_train; 
    CREATE TABLE application_train(
        SK_ID_CURR integer,
        TARGET boolean,
        NAME_CONTRACT_TYPE varchar(17),
        CODE_GENDER varchar(3),
        FLAG_OWN_CAR char(1),
        FLAG_OWN_REALTY char(1),
        CNT_CHILDREN real,
        AMT_INCOME_TOTAL real,
        AMT_CREDIT real,
        AMT_ANNUITY real,
        AMT_GOODS_PRICE real,
        NAME_TYPE_SUITE varchar(17),
        NAME_INCOME_TYPE varchar(25),
        NAME_EDUCATION_TYPE varchar(30),
        NAME_FAMILY_STATUS varchar(26),
        NAME_HOUSING_TYPE varchar(25),
        REGION_POPULATION_RELATIVE real,
        DAYS_BIRTH integer,
        DAYS_EMPLOYED integer,
        DAYS_REGISTRATION real,
        DAYS_ID_PUBLISH smallint,
        OWN_CAR_AGE real,
        FLAG_MOBIL boolean,
        FLAG_EMP_PHONE boolean,
        FLAG_WORK_PHONE boolean,
        FLAG_CONT_MOBILE boolean,
        FLAG_PHONE boolean,
        FLAG_EMAIL boolean,
        OCCUPATION_TYPE varchar(40),
        CNT_FAM_MEMBERS real,
        REGION_RATING_CLIENT smallint, 
        REGION_RATING_CLIENT_W_CITY smallint,
        WEEKDAY_APPR_PROCESS_START varchar(9),
        HOUR_APPR_PROCESS_START smallint,
        REG_REGION_NOT_LIVE_REGION boolean,
        REG_REGION_NOT_WORK_REGION boolean,
        LIVE_REGION_NOT_WORK_REGION boolean,
        REG_CITY_NOT_LIVE_CITY boolean,
        REG_CITY_NOT_WORK_CITY boolean,
        LIVE_CITY_NOT_WORK_CITY boolean,
        ORGANIZATION_TYPE varchar(30), 
        EXT_SOURCE_1 real,
        EXT_SOURCE_2 real,
        EXT_SOURCE_3 real,
        APARTMENTS_AVG real,
        BASEMENTAREA_AVG real,
        YEARS_BEGINEXPLUATATION_AVG real,
        YEARS_BUILD_AVG real,
        COMMONAREA_AVG real,
        ELEVATORS_AVG real,
        ENTRANCES_AVG real,
        FLOORSMAX_AVG real,
        FLOORSMIN_AVG real,
        LANDAREA_AVG real,
        LIVINGAPARTMENTS_AVG real,
        LIVINGAREA_AVG real,
        NONLIVINGAPARTMENTS_AVG real,
        NONLIVINGAREA_AVG real,
        APARTMENTS_MODE real,
        BASEMENTAREA_MODE real,
        YEARS_BEGINEXPLUATATION_MODE real,
        YEARS_BUILD_MODE real,
        COMMONAREA_MODE real,
        ELEVATORS_MODE real,
        ENTRANCES_MODE real,
        FLOORSMAX_MODE real,
        FLOORSMIN_MODE real,
        LANDAREA_MODE real,
        LIVINGAPARTMENTS_MODE real,
        LIVINGAREA_MODE real,
        NONLIVINGAPARTMENTS_MODE real,
        NONLIVINGAREA_MODE real,
        APARTMENTS_MEDI real,
        BASEMENTAREA_MEDI real,
        YEARS_BEGINEXPLUATATION_MEDI real,
        YEARS_BUILD_MEDI real,
        COMMONAREA_MEDI real,
        ELEVATORS_MEDI real,
        ENTRANCES_MEDI real,
        FLOORSMAX_MEDI real,
        FLOORSMIN_MEDI real,
        LANDAREA_MEDI real,
        LIVINGAPARTMENTS_MEDI real,
        LIVINGAREA_MEDI real,
        NONLIVINGAPARTMENTS_MEDI real,
        NONLIVINGAREA_MEDI real,
        FONDKAPREMONT_MODE varchar(25),
        HOUSETYPE_MODE varchar(17),
        TOTALAREA_MODE real,
        WALLSMATERIAL_MODE varchar(13),
        EMERGENCYSTATE_MODE varchar(3),
        OBS_30_CNT_SOCIAL_CIRCLE real,
        DEF_30_CNT_SOCIAL_CIRCLE real,
        OBS_60_CNT_SOCIAL_CIRCLE real,
        DEF_60_CNT_SOCIAL_CIRCLE real,
        DAYS_LAST_PHONE_CHANGE real,
        FLAG_DOCUMENT_2 boolean,
        FLAG_DOCUMENT_3 boolean,
        FLAG_DOCUMENT_4 boolean,
        FLAG_DOCUMENT_5 boolean,
        FLAG_DOCUMENT_6 boolean,
        FLAG_DOCUMENT_7 boolean,
        FLAG_DOCUMENT_8 boolean,
        FLAG_DOCUMENT_9 boolean,
        FLAG_DOCUMENT_10 boolean,
        FLAG_DOCUMENT_11 boolean,
        FLAG_DOCUMENT_12 boolean,
        FLAG_DOCUMENT_13 boolean,
        FLAG_DOCUMENT_14 boolean,
        FLAG_DOCUMENT_15 boolean,
        FLAG_DOCUMENT_16 boolean,
        FLAG_DOCUMENT_17 boolean,
        FLAG_DOCUMENT_18 boolean,
        FLAG_DOCUMENT_19 boolean,
        FLAG_DOCUMENT_20 boolean,
        FLAG_DOCUMENT_21 boolean,
        AMT_REQ_CREDIT_BUREAU_HOUR real,
        AMT_REQ_CREDIT_BUREAU_DAY real,
        AMT_REQ_CREDIT_BUREAU_WEEK real,
        AMT_REQ_CREDIT_BUREAU_MON real,
        AMT_REQ_CREDIT_BUREAU_QRT real,
        AMT_REQ_CREDIT_BUREAU_YEAR real)
"""

sql_data_query = f"""
COPY application_train(
    SK_ID_CURR,
    TARGET,
    NAME_CONTRACT_TYPE,
    CODE_GENDER,
    FLAG_OWN_CAR,
    FLAG_OWN_REALTY,
    CNT_CHILDREN,
    AMT_INCOME_TOTAL,
    AMT_CREDIT,
    AMT_ANNUITY,
    AMT_GOODS_PRICE,
    NAME_TYPE_SUITE,
    NAME_INCOME_TYPE,
    NAME_EDUCATION_TYPE,
    NAME_FAMILY_STATUS,
    NAME_HOUSING_TYPE,
    REGION_POPULATION_RELATIVE,
    DAYS_BIRTH,
    DAYS_EMPLOYED,
    DAYS_REGISTRATION,
    DAYS_ID_PUBLISH,
    OWN_CAR_AGE,
    FLAG_MOBIL,
    FLAG_EMP_PHONE,
    FLAG_WORK_PHONE,
    FLAG_CONT_MOBILE,
    FLAG_PHONE,
    FLAG_EMAIL,
    OCCUPATION_TYPE,
    CNT_FAM_MEMBERS,
    REGION_RATING_CLIENT,
    REGION_RATING_CLIENT_W_CITY,
    WEEKDAY_APPR_PROCESS_START,
    HOUR_APPR_PROCESS_START,
    REG_REGION_NOT_LIVE_REGION,
    REG_REGION_NOT_WORK_REGION,
    LIVE_REGION_NOT_WORK_REGION,
    REG_CITY_NOT_LIVE_CITY,
    REG_CITY_NOT_WORK_CITY,
    LIVE_CITY_NOT_WORK_CITY,
    ORGANIZATION_TYPE,
    EXT_SOURCE_1,
    EXT_SOURCE_2,
    EXT_SOURCE_3,
    APARTMENTS_AVG,
    BASEMENTAREA_AVG,
    YEARS_BEGINEXPLUATATION_AVG,
    YEARS_BUILD_AVG,
    COMMONAREA_AVG,
    ELEVATORS_AVG,
    ENTRANCES_AVG,
    FLOORSMAX_AVG,
    FLOORSMIN_AVG,
    LANDAREA_AVG,
    LIVINGAPARTMENTS_AVG,
    LIVINGAREA_AVG,
    NONLIVINGAPARTMENTS_AVG,
    NONLIVINGAREA_AVG,
    APARTMENTS_MODE,
    BASEMENTAREA_MODE,
    YEARS_BEGINEXPLUATATION_MODE,
    YEARS_BUILD_MODE,
    COMMONAREA_MODE,
    ELEVATORS_MODE,
    ENTRANCES_MODE,
    FLOORSMAX_MODE,
    FLOORSMIN_MODE,
    LANDAREA_MODE,
    LIVINGAPARTMENTS_MODE,
    LIVINGAREA_MODE,
    NONLIVINGAPARTMENTS_MODE,
    NONLIVINGAREA_MODE,
    APARTMENTS_MEDI,
    BASEMENTAREA_MEDI,
    YEARS_BEGINEXPLUATATION_MEDI,
    YEARS_BUILD_MEDI,
    COMMONAREA_MEDI,
    ELEVATORS_MEDI,
    ENTRANCES_MEDI,
    FLOORSMAX_MEDI,
    FLOORSMIN_MEDI,
    LANDAREA_MEDI,
    LIVINGAPARTMENTS_MEDI,
    LIVINGAREA_MEDI,
    NONLIVINGAPARTMENTS_MEDI,
    NONLIVINGAREA_MEDI,
    FONDKAPREMONT_MODE,
    HOUSETYPE_MODE,
    TOTALAREA_MODE,
    WALLSMATERIAL_MODE,
    EMERGENCYSTATE_MODE,
    OBS_30_CNT_SOCIAL_CIRCLE,
    DEF_30_CNT_SOCIAL_CIRCLE,
    OBS_60_CNT_SOCIAL_CIRCLE,
    DEF_60_CNT_SOCIAL_CIRCLE,
    DAYS_LAST_PHONE_CHANGE,
    FLAG_DOCUMENT_2,
    FLAG_DOCUMENT_3,
    FLAG_DOCUMENT_4,
    FLAG_DOCUMENT_5,
    FLAG_DOCUMENT_6,
    FLAG_DOCUMENT_7,
    FLAG_DOCUMENT_8,
    FLAG_DOCUMENT_9,
    FLAG_DOCUMENT_10,
    FLAG_DOCUMENT_11,
    FLAG_DOCUMENT_12,
    FLAG_DOCUMENT_13,
    FLAG_DOCUMENT_14,
    FLAG_DOCUMENT_15,
    FLAG_DOCUMENT_16,
    FLAG_DOCUMENT_17,
    FLAG_DOCUMENT_18,
    FLAG_DOCUMENT_19,
    FLAG_DOCUMENT_20,
    FLAG_DOCUMENT_21,
    AMT_REQ_CREDIT_BUREAU_HOUR,
    AMT_REQ_CREDIT_BUREAU_DAY,
    AMT_REQ_CREDIT_BUREAU_WEEK,
    AMT_REQ_CREDIT_BUREAU_MON,
    AMT_REQ_CREDIT_BUREAU_QRT,
    AMT_REQ_CREDIT_BUREAU_YEAR
)
FROM '{DATA_FULL_PATH}application_train.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
postgres_db.send_query(sql_schema_query)

In [ ]:
%%time
postgres_db.send_query(sql_data_query)

---
1. Вывести средний доход среди всех клиентов
---

In [ ]:
%%time

query = """
SELECT AVG(amt_income_total)
FROM (
    SELECT amt_income_total FROM application_test
    UNION 
    SELECT amt_income_total FROM application_train
) union_application               

"""
postgres_db.get_query(query)

---
2. Вывести минимальный и максимальный возраст среди всех клиентов
---

In [ ]:
%%time

query = """
SELECT ABS(MIN(days_birth) / 365) AS Maximum_age, ABS(MAX(days_birth) / 365) AS Minimum_age
FROM (
    SELECT days_birth FROM application_test
    UNION 
    SELECT days_birth FROM application_train
) union_application               

"""
postgres_db.get_query(query)

---
3. Вывести количество мужчин и женщин
---

In [ ]:
%%time

query = """
SELECT code_gender, count(code_gender)
FROM (
    SELECT sk_id_curr, code_gender FROM application_test
    UNION 
    SELECT sk_id_curr, code_gender FROM application_train
) union_application
GROUP BY code_gender
HAVING code_gender='M' or code_gender='F'
"""
postgres_db.get_query(query)

---
4. Вывести общую сумму, количество и среднюю сумму, запрошенную клиентами в кредит с авто и без
---

In [ ]:
%%time

query = """
SELECT flag_own_car, sum(amt_credit), count(amt_credit), avg(amt_credit)
FROM (
    SELECT sk_id_curr, flag_own_car, amt_credit FROM application_test
    UNION 
    SELECT sk_id_curr, flag_own_car, amt_credit FROM application_train
) union_application
GROUP BY flag_own_car
"""
postgres_db.get_query(query)

---
5. Вывести доли клиентов с различным образованием
---

In [ ]:
%%time
# NAME_EDUCATION_TYPE

query = """
SELECT name_education_type, count_education::real / all_count as share
FROM
(
    SELECT name_education_type, COUNT(sk_id_curr) as count_education
    FROM (
        SELECT sk_id_curr, name_education_type FROM application_test
        UNION 
        SELECT sk_id_curr, name_education_type FROM application_train
    ) union_application
    GROUP BY name_education_type
) groups_education
CROSS JOIN
(
    SELECT COUNT(sk_id_curr) as all_count
    FROM (
        SELECT sk_id_curr, name_education_type FROM application_test
        UNION 
        SELECT sk_id_curr, name_education_type FROM application_train
    ) counter
) count_all

"""
postgres_db.get_query(query)

---
6. Подсчитать количество полных лет для клиентов, у которых есть во владении автомобиль и недвижимость. Вывести топ 10 по возрастанию
---

In [ ]:
%%time

# НУЖЕН ЛИ DISTINCT

query = """
SELECT DISTINCT ABS(days_birth / 365) as full_years
FROM (
    SELECT sk_id_curr, flag_own_car, flag_own_realty, days_birth FROM application_test
    UNION 
    SELECT sk_id_curr, flag_own_car, flag_own_realty, days_birth FROM application_train
) union_application
WHERE flag_own_realty='Y' and flag_own_car='Y'
ORDER BY full_years
LIMIT 10
"""
postgres_db.get_query(query)

---
7. Вывести тех клиентов, у кого доход на одного члена семьи в два раза больше, чем в среднем на одного члена семьи по выборке
---

In [ ]:
%%time

query = """
SELECT sk_id_curr, cnt_fam_members / amt_income_total as income_per_member, avg_income, (cnt_fam_members / amt_income_total) / avg_income as ratio
FROM (
    SELECT sk_id_curr, amt_income_total, cnt_fam_members FROM application_test
    UNION 
    SELECT sk_id_curr, amt_income_total, cnt_fam_members FROM application_train
) union_application
CROSS JOIN
(
SELECT AVG(cnt_fam_members / amt_income_total) as avg_income
FROM (
    SELECT sk_id_curr, amt_income_total, cnt_fam_members FROM application_test
    UNION 
    SELECT sk_id_curr, amt_income_total, cnt_fam_members FROM application_train
    ) union_avg_income
) union_avg_income
WHERE (cnt_fam_members / amt_income_total) / avg_income > 2
"""
postgres_db.get_query(query)

---
8. Вывести клиентов старше 60 лет по которым нет данных в bureau
---

In [ ]:
%%time

query = """
SELECT sk_id_curr
    FROM (
        SELECT sk_id_curr, flag_own_car, flag_own_realty, days_birth FROM application_test
        UNION 
        SELECT sk_id_curr, flag_own_car, flag_own_realty, days_birth FROM application_train
    ) union_application
    WHERE ABS(days_birth / 365) > 60
    EXCEPT SELECT sk_id_curr
            FROM bureau
"""
postgres_db.get_query(query)

---
9. Вывести женщин, у которых в истории bureau было больше двух кредитов, просроченных на 61 день и более. Отсортировать в порядке убывания по кол-ву таких кредитов
---

In [ ]:
%%time

query = """
SELECT union_application.sk_id_curr
    FROM (
        SELECT sk_id_curr, code_gender FROM application_test
        WHERE code_gender='F'
        UNION 
        SELECT sk_id_curr, code_gender FROM application_train
        WHERE code_gender='F'
    ) union_application
JOIN 
    (SELECT sk_id_curr, count(sk_id_bureau) as amount_credits -- Клиенты, которые имеют более 2-х просроченных кредитов
         FROM bureau
         WHERE CREDIT_DAY_OVERDUE > 60
         GROUP BY sk_id_curr
         HAVING count(sk_id_bureau) > 2
    ) debtors ON union_application.sk_id_curr = debtors.sk_id_curr
ORDER BY debtors.amount_credits DESC

"""
postgres_db.get_query(query)

---
10. По данным из bureau (БКИ) расчитать долю просрочки в активных займах для каждого клиента. Вывести топ 7 мужчин с наибольшей суммой просрочки, указав для них, помимо прочего, сумму активных кредитов и суммы всех кредитов (активных и закрытых).
---

In [ ]:
%%time

query = """
SELECT bureau_table.sk_id_curr, bureau_table.overdue_amount, bureau_table.percent_overdue, bureau_table.active_credit_sum, bureau_table.all_credit_sum
FROM 
    (SELECT main_query.sk_id_curr, main_query.overdue_amount, main_query.percent_overdue, main_query.active_credit_sum, all_credit.all_credit_sum
    FROM 
        (SELECT sk_id_curr, SUM(amt_credit_sum_debt) AS overdue_amount, SUM(amt_credit_sum_debt/amt_credit_sum) AS percent_overdue, SUM(amt_credit_sum) as active_credit_sum
        FROM bureau
        WHERE credit_active='Active' AND amt_credit_sum_debt > 0 AND amt_credit_sum > 0
        GROUP BY sk_id_curr, amt_credit_sum_debt / amt_credit_sum
        ) main_query
    JOIN
    (SELECT sk_id_curr, SUM(amt_credit_sum) AS all_credit_sum
        FROM bureau
        GROUP BY sk_id_curr
    ) all_credit ON all_credit.sk_id_curr = main_query.sk_id_curr
    ) bureau_table
JOIN 
    (SELECT sk_id_curr 
    FROM (SELECT sk_id_curr, code_gender FROM application_test
            WHERE code_gender='M'
            UNION 
            SELECT sk_id_curr, code_gender FROM application_train
            WHERE code_gender='M') application_union
    ) application_table ON application_table.sk_id_curr=bureau_table.sk_id_curr
ORDER BY bureau_table.overdue_amount DESC
LIMIT 7
"""
postgres_db.get_query(query)

In [ ]:
%%time

query_test = """
SELECT sk_id_curr, cnt_fam_members / amt_income_total as income_per_member
FROM application_test
"""

query_train = """
SELECT sk_id_curr, cnt_fam_members / amt_income_total as income_per_member
FROM application_train
"""

postgres_db.get_query(query_test)

In [ ]:
postgres_db.close()